In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.utils import shuffle

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'19lK2ZuvmN6LeHfpuYXd2tBNUD34rndFb'})
downloaded.GetContentFile('fer2013.csv')

In [ ]:
def getData(balance_ones = True , Ntest= 1000):
  X =[]
  Y =[]
  first = True
  for line in open('fer2013.csv'):
    if first:
      first = False
    else:
      row = line.split(',')
      Y.append(int(row[0]))
      X.append([int(p) for p in row[1].split()])
  
  X,Y = np.array(X)/255.0 , np.array(Y)
  X , Y = shuffle(X,Y)
  Xtrain , Ytrain = X[:-Ntest] , Y[:-Ntest]
  Xvalid , Yvalid = X[-Ntest:] , Y[-Ntest:]
  if balance_ones:
    X0,Y0 = Xtrain[Ytrain!=1, :] , Ytrain[Ytrain!=1]
    X1 = Xtrain[Ytrain==1, :]
    X1 = np.repeat(X1 , 9 , axis=0)
    Xtrain = np.vstack([X0,X1])
    Ytrain = np.concatenate((Y0 , [1]*len(X1)))
    return Xtrain , Ytrain , Xvalid , Yvalid

In [ ]:
def getImageData():
  Xtrain , Ytrain , Xvalid , Yvalid = getData()
  N,D = Xtrain.shape
  d = int(np.sqrt(D))
  Xtrain = Xtrain.reshape(-1,d,d,1)
  Xvalid = Xvalid.reshape(-1,d,d,1)
  return Xtrain , Ytrain , Xvalid , Yvalid

In [ ]:
def error_rate(targets , predictions):
  return np.mean(targets != predictions)

In [ ]:
def init_weight_and_bias(M1,M2):
  w = np.random.randn(M1,M2) / np.sqrt(M1)
  b = np.zeros(M2)
  return w.astype(np.float32) , b.astype(np.float32)

In [ ]:
def y2indicator(y):
  N = len(y)
  K = len(set(y))
  ind = np.zeros((N,K))
  for i in range(N):
    ind[i , y[i]] = 1
  return ind

In [ ]:
class HiddenLayer(object):
  def __init__(self,M1,M2,an_id):
    self.id = an_id
    self.M1 = M1
    self.M2 = M2
    w,b = init_weight_and_bias(M1,M2)
    self.w = tf.Variable(w.astype(np.float32))
    self.b = tf.Variable(b.astype(np.float32))
    self.params = [self.w , self.b]

  def forward(self,X):
    return tf.nn.relu(tf.matmul(X, self.w) + self.b)

In [ ]:
def init_filter(shape, poolsz):
  w = np.random.randn(*shape) *np.sqrt(2) / np.sqrt(np.prod(shape[:-1]) +shape[-1]*np.prod(shape[:-2] / np.prod(poolsz)))
  return w.astype(np.float32)

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(48, 48, 1)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(7, activation='softmax')])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 21, 21, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)             

In [ ]:
Xtrain , Ytrain , Xvalid , Yvalid = getImageData()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modl = model.fit(x=Xtrain, y=Ytrain, epochs=15)

Epoch 1/15
1224/1224 [==============================] - 70s 56ms/step - loss: 1.7636 - accuracy: 0.3057
Epoch 2/15
1224/1224 [==============================] - 69s 56ms/step - loss: 1.2881 - accuracy: 0.5109
Epoch 3/15
1224/1224 [==============================] - 69s 56ms/step - loss: 1.0793 - accuracy: 0.5905
Epoch 4/15
1224/1224 [==============================] - 68s 56ms/step - loss: 0.9553 - accuracy: 0.6401
Epoch 5/15
1224/1224 [==============================] - 68s 56ms/step - loss: 0.8586 - accuracy: 0.6769
Epoch 6/15
1224/1224 [==============================] - 69s 56ms/step - loss: 0.7847 - accuracy: 0.7053
Epoch 7/15
1224/1224 [==============================] - 68s 56ms/step - loss: 0.6881 - accuracy: 0.7469
Epoch 8/15
1224/1224 [==============================] - 68s 55ms/step - loss: 0.5907 - accuracy: 0.7852
Epoch 9/15
1224/1224 [==============================] - 68s 55ms/step - loss: 0.4918 - accuracy: 0.8241
Epoch 10/15
1224/1224 [==============================] - 67s 55m

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(48, 48))
  x = image.img_to_array(img)
  x = x / 255
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  
 

Saving images.jfif to images.jfif


ValueError: ignored